<a href="https://colab.research.google.com/github/RatchanonMo/curriculum/blob/main/data_gathering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#fastai
!pip install -q fastbook
from fastbook import *
from fastai.vision.widgets import *
from tqdm.auto import tqdm

#download from duckduckgo
def search_images_ddg(key, max_n=200):
     """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
        (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
     """
     url        = 'https://duckduckgo.com/'
     params     = {'q':key}
     res        = requests.post(url,data=params)
     searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
     if not searchObj: print('Token Parsing Failed !'); return
     requestUrl = url + 'i.js'
     headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
     params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
     urls       = []
     while True:
         try:
             res  = requests.get(requestUrl,headers=headers,params=params)
             data = json.loads(res.text)
             for obj in data['results']:
                 urls.append(obj['image'])
                 max_n = max_n - 1
                 if max_n < 1: return L(set(urls))     # dedupe
             if 'next' not in data: return L(set(urls))
             requestUrl = url + data['next']
         except:
             pass

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
#save ไว้ที่ไหน
path = Path('dataset')
#เอารูปอะไรบ้าง (labels)
labels = ['Apple__healthy_leaf','Apple__scab_leaf','Apple__Black_rot_leaf','Apple__Cedar_apple_rust_leaf']
#เอา label ละกี่รูป
MAX_N = 1500

if not path.exists():
    path.mkdir()
    for l in tqdm(labels):
        dest = (path/l)
        dest.mkdir(exist_ok=True)
        results = search_images_ddg(l, max_n=MAX_N)
        download_images(dest, urls=results)

  0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
#เช็คว่าได้รูปอะไรบ้าง
fns = get_image_files(path)
fns

(#2152) [Path('dataset/Apple__scab_leaf/00000281.jpg'),Path('dataset/Apple__scab_leaf/00000429.jpg'),Path('dataset/Apple__scab_leaf/00000441.JPG'),Path('dataset/Apple__scab_leaf/00000501.jpg'),Path('dataset/Apple__scab_leaf/00000271.jpg'),Path('dataset/Apple__scab_leaf/00000319.jpg'),Path('dataset/Apple__scab_leaf/00000468.jpg'),Path('dataset/Apple__scab_leaf/00000024.png'),Path('dataset/Apple__scab_leaf/00000398.jpg'),Path('dataset/Apple__scab_leaf/00000066.jpg')...]

In [25]:
#ดูว่าหารูปไหนไม่เจอบ้าง
failed = verify_images(fns)
failed

(#47) [Path('dataset/Apple__scab_leaf/00000034.jpg'),Path('dataset/Apple__scab_leaf/00000144.jpg'),Path('dataset/Apple__scab_leaf/00000161.jpg'),Path('dataset/Apple__scab_leaf/00000119.jpg'),Path('dataset/Apple__scab_leaf/00000134.jpg'),Path('dataset/Apple__scab_leaf/00000002.jpg'),Path('dataset/Apple__scab_leaf/00000539.png'),Path('dataset/Apple__Cedar_apple_rust_leaf/00000432.jpg'),Path('dataset/Apple__Cedar_apple_rust_leaf/00000039.jpg'),Path('dataset/Apple__Cedar_apple_rust_leaf/00000486.jpg')...]

In [26]:
#นำรูปที่หาไม่เจอออก
failed.map(Path.unlink)

FileNotFoundError: ignored

**Kaggle**

In [28]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
%cd /content/drive/MyDrive/plant_diseases_classification/data

In [ ]:
!unzip new-plant-diseases-dataset.zip 